# Tool을 사용하는 챗봇 구현

## OpenAI LLM 준비
* 환경 변수(`.env` 파일)에서 API Key 로딩
* 개발 환경에서는 `gpt-4o-mini` 또는 `gpt-3.5-turbo`

In [ ]:
import os
import gradio as gr
from dotenv import load_dotenv
from typing import Any

from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.tools import tool

from langchain_core.messages import AIMessage


# .env 파일 로드 및 API 키 확인
load_dotenv()

# OpenAI LLM 준비
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.7)

## 데코레이터를 통한 Tool 구성
코난이 사용할 수 있는 도구 추천 툴 함수
- `@tool` 데코레이터(Decorator): 함수의 이름, 설명, 그리고 입출력 정보를 자동으로 추출하여 LLM에게 전달
- 함수의 이름과 따옴표 세 개("""...""")로 감싸인 주석(Docstring)을 읽고 이 도구의 용도를 파악
- `"마취 시계: ..."` : 도구가 언제, 어떤 상황에 필요한지를 설명
- `Args`: 도구를 사용하는 데 어떤 정보가 필요한지를 설명 - LLM은 이 부분을 보고 사용자의 요청에서 필요한 정보를 target이라는 인자로 추출해서 전달
- `return` 값: 이 함수가 실제로 실행되었을 때의 결과 데이터

In [ ]:
## 단계 - 도구 구성
@tool
def tranquilizer_watch(target: str) -> str:
    """마취 시계: 지정된 대상을 잠재울 필요가 있을 때 사용합니다. 추리 설명 등을 대신할 때 유용합니다.
    Args:
        target (str): 마취시킬 대상의 이름이나 인상착의. 예: '안경 쓴 범인', '유명한 탐정님'
    """
    return f"⌚ 마취 시계: '{target}'을(를) 성공적으로 마취시켰습니다."


@tool
def voice_changer_bowtie(target: str) -> str:
    """음성 변조 나비넥타이: 다른 사람의 목소리로 추리를 설명하거나, 다른 사람인 척 연기해야 할 때 사용합니다.
    Args:
        target (str): 목소리를 흉내 낼 대상. 예: '브라운 박사님', '유명한 탐정님'
    """
    return f"🎤 음성 변조 나비넥타이: '{target}'의 목소리로 변조를 시작합니다."


@tool
def detective_glasses(target: str) -> str:
    """탐정 안경: 특정 대상을 추적하거나 멀리 있는 것을 확대해서 볼 때 사용합니다. 범인 추적에 필수적입니다.
    Args:
        target (str): 추적하거나 확대할 대상. 예: '범인의 자동차', '먼 곳의 단서'
    """
    return f"🕶️ 탐정 안경: '{target}'에 대한 추적 및 확대 기능을 활성화합니다."


@tool
def soccer_shoes(target: str) -> str:
    """킥력 강화 축구화: 강력한 힘으로 무언가를 걷어차 범인을 제압하거나 위기 상황을 탈출할 때 사용합니다.
    Args:
        target (str): 강하게 찰 대상. 예: '범인을 위협할 돌멩이', '막다른 길의 문'
    """
    return f"⚽ 킥력 강화 축구화: '{target}'을(를) 향해 강력한 킥을 준비합니다!"

## LLM에 도구(tools) 연결하기
LLM이 어떤 도구들을 사용할 수 있는지, 그리고 각 도구가 어떤 기능을 하는지 인지할 수 있도록 바인딩(binding)

In [ ]:
# 도구 목록 정리
tools = [tranquilizer_watch, voice_changer_bowtie, detective_glasses, soccer_shoes]

# 도구 목록을 LLM에 연결
llm_with_tools = llm.bind_tools(tools)

## 프롬프트 구성
복잡한 상황에서 여러 도구를 조합하여 사용할 수 있도록 메시지 작성
- System(배경 설정): 나는 명탐정 코난이고, 주어진 상황을 해결하기 위해 도구를 사용해야 한다.
- Human(사용자 질문): {사용자가 입력한 상황}

In [ ]:
## 단계 - 프롬프트
prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 명탐정 코난입니다. 주어진 상황을 해결하기 위해 당신이 가진 도구들을 적절하게 사용하세요. 상황에 따라 여러 도구를 동시에 사용할 수도 있습니다."),
    ("human", "{situation}"),
])

## 도구 실행기 구성
LLM이 내린 '결정'을 입력으로 받아, 실제로 그에 해당하는 파이썬 함수를 찾아서 실행시키는 `도구 실행자`를 구성. 
- `tool_map = {tool.name: tool for tool in tools}`: tools 리스트에서 각 tool의 이름을 **키(key)** 로, 함수 객체를 **값(value)** 으로 하는 딕셔너리(dictionary)를 구성
- `def tool_executor(ai_message: AIMessage) -> Any`: LLM의 최종 출력물인 ai_message (AIMessage 타입)을 입력으로 받는 함수 정의
    - `ai_message.tool_calls` : LLM이 도구를 사용해야 한다고 판단하면 도구명이 전달됨

In [ ]:
# 실행할 도구를 이름으로 쉽게 찾을 수 있도록 맵을 생성
tool_map = {tool.name: tool for tool in tools}


# 실행 담당자 함수
def tool_executor(ai_message: AIMessage) -> Any:
    """LLM의 출력을 받아, tool_calls가 있으면 해당 도구를 실행하고 결과를 반환합니다."""
    
    # 디버깅을 위한 출력
    print(ai_message)

    # LLM이 도구를 사용하지 않고 일반 메시지로 답변한 경우, 내용을 그대로 반환
    if not isinstance(ai_message, AIMessage) or not ai_message.tool_calls:
        return ai_message.content

    # Tool 호출 결과를 저장할 배열
    results = []

    # LLM이 호출한 모든 도구를 순차적으로 실행
    for tool_call in ai_message.tool_calls:
        tool_to_run = tool_map.get(tool_call["name"])
        if tool_to_run:
            observation = tool_to_run.invoke(tool_call["args"])
            results.append(observation)
        else:
            results.append(f"오류: '{tool_call['name']}' 도구를 찾을 수 없습니다.")

    # results 배열의 항목을 문자형으로 변환 후 개행문자로 연결하여 전달
    return "\n".join(str(res) for res in results)

## LCEL 파이프라인 구성
- `prompt`: AI에게 역할을 부여하고 사용자 입력을 형식에 맞게 가공하는 '지령서'
- `llm_with_tools`: 도구를 사용할 줄 아는 'AI Agent'
- `tool_executor`: AI의 결정을 실제 행동으로 옮기는 '실행 담당자

In [ ]:
# LCEL 파이프라인 구성
chain = prompt | llm_with_tools | tool_executor

## Gradio UI 구성
### Gradio 처리 함수 설정

In [ ]:
def chat_fn(user_input: str) -> str:
    
    if not user_input:
        return "상황을 입력해주세요."

    return chain.invoke({"situation": user_input})

### Gradio UI 구성

In [ ]:
with gr.Blocks() as demo:
    gr.Markdown("### 🕵️ 명탐정 코난 도구 추천기")
    gr.Markdown("상황을 입력하면 코난이 상황에 적절한 도구를 사용하게 됩니다.")

    user_input = gr.Textbox(label="상황 설명", placeholder="예: 모리 탐정을 기절시키고, 모리 탐정 목소리로 사건을 설명하고 싶어요")
    ai_output = gr.Textbox(label="추천 도구", lines=5)
    user_input.submit(chat_fn, inputs=user_input, outputs=ai_output)

demo.launch()

-----
** End of Documents **